# 1. background

* 获取到了“预测值”：weights/test_df_fastrcnn.jl
* 获取到了“真实值”：weights/valid_df_fastrcnn.jl


# 2. load data

In [97]:
import pandas as pd
import os

d = '1'

df_pred = pd.read_excel(  os.path.join(d, 'test_df_fastrcnn.jl.xlsx')  )
df_true = pd.read_excel(  os.path.join(d, 'valid_df_fastrcnn.jl.xlsx')  )

print('df_pred',df_pred.shape)
print('df_true',df_true.shape)

df_pred (9926, 9)
df_true (10426, 10)


In [98]:
df_pred.head(3)

,Unnamed: 0,score,x_min,y_min,x_max,y_max,label,pathname,category
0,0,0.998232,623,1013,739,1130,1,images/0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png,red blood cell
1,1,0.998082,1447,716,1582,846,1,images/0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png,red blood cell
2,2,0.997781,864,334,997,453,1,images/0dcca702-a4ef-4fb3-a940-9c0c140b21c7.png,red blood cell


In [99]:
pred_image_to_loc = {}

for index, row in df_pred.iterrows():
    pathname = row['pathname']
    image_name = pathname.replace('images/', '')
    x_min = row['x_min']
    y_min = row['y_min']
    x_max = row['x_max']
    y_max = row['y_max']
    c = [x_min,y_min,x_max,y_max]
    if image_name not in pred_image_to_loc:
        pred_image_to_loc[image_name] = []
    pred_image_to_loc[image_name].append(c)
    
print('pred_image_to_loc len = ', len(pred_image_to_loc) )

pred_image_to_loc len =  160


In [100]:
true_image_to_loc = {}

for index, row in df_true.iterrows():
    pathname = row['pathname']
    image_name = pathname.replace('images/', '')
    x_min = row['x_min']
    y_min = row['y_min']
    x_max = row['x_max']
    y_max = row['y_max']
    c = [x_min,y_min,x_max,y_max]
    if image_name not in true_image_to_loc:
        true_image_to_loc[image_name] = []
    true_image_to_loc[image_name].append(c)
    
print('true_image_to_loc len = ', len(true_image_to_loc) )

true_image_to_loc len =  160


# 3. IOU

In [101]:
def get_iou(c1, c2):
    x1_min,y1_min,x1_max,y1_max = c1
    x2_min,y2_min,x2_max,y2_max = c2
    left_column_max = max(x1_min,x2_min)
    right_column_min = min(x1_max,x2_max)
    up_row_max = max(y1_min,y2_min)
    down_row_min = min(y1_max,y2_max)
    if left_column_max>=right_column_min or up_row_max>=down_row_min:
        return 0
        
    area_cross = (right_column_min-left_column_max)*(down_row_min-up_row_max)
    area_s1 = (x1_max-x1_min)*(y1_max-y1_min)
    area_s2= (x2_max-x2_min)*(y2_max-y2_min)
    return area_cross/(area_s1+area_s2-area_cross)

In [102]:
def match_cells(pred_cells, true_cells):
    pred_correct_index_list = []
    for j in range(len(pred_cells)):
        c1 = pred_cells[j]
        iou_list = []
        for i in range(len(true_cells)):
            c2 = true_cells[i]
            iou = get_iou(c1, c2)
            iou_list.append(iou)
        max_value_index = 0
        max_value = iou_list[max_value_index]
        for i in range(len(iou_list)):
            v = iou_list[i]
            if v>max_value:
                max_value_index = i
                max_value = iou_list[max_value_index]
        if max_value>0.8:
            pred_correct_index_list.append(max_value_index)
        else:
            pred_correct_index_list.append(-1)
    return pred_correct_index_list            

In [103]:
def get_pred_correct_count(pred_cell_list, true_cell_list):
    pred_correct_index_list = match_cells(pred_cell_list, true_cell_list)
    tmp = []
    for x in pred_correct_index_list:
        if x!=-1:
            tmp.append(x)
    tmp = list(set(tmp))
    return len(tmp)

In [104]:
def get_acc(pred_image_to_loc, true_image_to_loc):
    true_sum = 0
    acc_sum = 0
    for image_name in pred_image_to_loc:
        pred_cell_list = pred_image_to_loc[image_name]
        true_cell_list = true_image_to_loc[image_name]
        true_sum += len(true_cell_list)
        correct_count = get_pred_correct_count(pred_cell_list, true_cell_list)
        acc_sum += correct_count
        #print('image_name={0}, true={1}, pred={2}'.format(image_name, len(true_cell_list), correct_count))
    return acc_sum/true_sum
acc = get_acc(pred_image_to_loc, true_image_to_loc)
print('acc=',acc)

acc= 0.7823709955879532


```
test, acc
weights_0405_epoch_1, 0.874640322271245
weights_0405_epoch_2, 0.874640322271245
weights_0405_epoch_5, 0.874640322271245
weights_0405_epoch_20, 0.874640322271245
```

test,acc
weights_0503_epoch_1,  acc= 0.125071935545751
weights_0503_epoch_3,  acc= 0.7567619413005947
weights_0503_epoch_5,  acc= 0.7905236907730673
weights_0503_epoch_10, acc= 0.7386341837713408


In [171]:
0529 k=10 epoch=1
fold0:acc= 0.13994762874599942
fold1:acc= 0.49254449254449256
fold2:acc= 0.386871237133424
fold3 acc= 0.2619369804755461
fold4 acc= 0.373522000387672
fold5 
    

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1376448020.py, line 1)

In [ ]:
0529 k=5 epoch=1
fold0 acc= 0.18285980437820215
fold1 acc= 0.505237097695677

In [ ]:
0529 k=10 epoch=3
fold0:acc= 0.7373678595674522
fold1:acc= 0.68003848003848

In [ ]:
0525
fold0 acc= 0.25599462881258395
fold1 acc= 0.46873201611356224
fold2 acc= 0.27479378476884714

